# 0. 초기 세팅

In [ ]:
import os
import cv2
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Pytorch에서 gpu를 사용하는 방법.
device = torch.device("cuda:4" if torch.cuda.is_available() else "cpu")
torch.cuda.set_device(device)
# print('Current cuda device ', torch.cuda.current_device())

In [ ]:
import PIL

In [ ]:
from matplotlib import pyplot as plt
import tifffile as tiff
from PIL import Image
import random

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.metrics import MeanIoU

In [ ]:
from keras.utils import normalize
from keras.metrics import MeanIoU

In [ ]:
from keras import backend as K

In [ ]:
import tensorflow as tf
import segmentation_models as sm
import segmentation_models_pytorch as smp
from segmentation_models_pytorch.encoders import get_preprocessing_fn

import glob
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt
import keras

from keras.utils import normalize
from keras.metrics import MeanIoU

In [ ]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "4"

In [ ]:
import random

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

In [ ]:
torch.cuda.memory_allocated()

In [ ]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.backends.cudnn.enabled)
print(torch.backends.cudnn.version())

# 성능 평가

**성능 평가를 위해 valid set에 대해서 결과 확인**

### 1) true_mask vs pred_mask 이미지 비교

In [ ]:
import numpy as np
import pandas as pd
from typing import List, Union
from joblib import Parallel, delayed

In [ ]:
import matplotlib.pyplot as plt

def display(display_list):
    plt.figure(figsize=(15, 15))

    title = ['Input Image', 'True Mask', 'Predicted Mask']

    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        plt.title(title[i])
        if i == 0:
            img = cv2.imread(display_list[i])
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = img.astype(np.uint8).copy()
        if i == 1:
            img = cv2.imread(display_list[i], 0)
        if i == 2:
#             img = rle_decode(display_list[i], shape = (224, 224)) # shape 설정
            img = display_list[i]

        plt.imshow(img)
        plt.axis('off')
    plt.show()

idx = 1000

valid_img_path_idx = valid_img_dir + df_valid['img_path'][idx]
valid_mask_path_idx = valid_mask_dir + df_valid['mask_path'][idx]
valid_pred_mask_idx = tmp_pred4[idx]

display_list = [valid_img_path_idx, valid_mask_path_idx, valid_pred_mask_idx]
display(display_list)

In [ ]:
weighted_preds_test = np.where(weighted_preds > 0.5, 1, 0)

In [ ]:
idx = 5

valid_img_path_idx = valid_img_dir + df_valid['img_path'][idx]
valid_mask_path_idx = valid_mask_dir + df_valid['mask_path'][idx]

In [ ]:
display_list = [valid_img_path_idx, valid_mask_path_idx, tmp_pred4[idx]]
display(display_list)

### 2) Dice Score

In [ ]:
def dice_score(prediction: np.array, ground_truth: np.array, smooth=1e-7) -> float:
    '''
    Calculate Dice Score between two binary masks.
    '''
    intersection = np.sum(prediction * ground_truth)
    return (2.0 * intersection + smooth) / (np.sum(prediction) + np.sum(ground_truth) + smooth)


def calculate_dice_scores(pred_mask, gt_mask, img_shape=(224, 224)) -> List[float]:
    '''
    Calculate Dice scores for a dataset.
    '''

    def calculate_dice(pred_mask, gt_mask):
        pred_mask = pred_mask
        gt_mask = gt_mask


        if np.sum(gt_mask) > 0 or np.sum(pred_mask) > 0:
            return dice_score(pred_mask, gt_mask)
        else:
            return None  # No valid masks found, return None


    dice_scores = Parallel(n_jobs=-1)(
        delayed(calculate_dice)(pred, gt) for pred, gt in zip(pred_mask, gt_mask)
    )


    dice_scores = [score for score in dice_scores if score is not None]  # Exclude None values


    return np.mean(dice_scores)

In [ ]:
calculate_dice_scores(tmp_pred4, valid_masks, img_shape=(224, 224))

### 3) Class 별 IOU

In [ ]:
valid_masks.shape

In [ ]:
import numpy as np

def calculate_iou(outputs, labels, num_classes):
    iou_per_class = np.zeros(num_classes)

    for class_idx in range(num_classes):
        # 클래스별 True Positive, False Positive, False Negative 계산
        true_positive = np.sum(np.logical_and(outputs == class_idx, labels == class_idx))
        false_positive = np.sum(np.logical_and(outputs == class_idx, labels != class_idx))
        false_negative = np.sum(np.logical_and(outputs != class_idx, labels == class_idx))

        # 클래스별 IoU 계산
        iou = true_positive / (true_positive + false_positive + false_negative + 1e-10)
        iou_per_class[class_idx] = iou

    return iou_per_class

# 예시 사용
outputs = tmp_pred4
labels = valid_masks
num_classes = 2
iou_per_class = calculate_iou(outputs, labels, num_classes)

print("클래스별 IoU:", iou_per_class)

In [ ]:
totaliou = iou_per_class

In [ ]:
def printClassScores(totaliou):
    label = ['background', 'building']
    print('classes          IoU      nIoU')
    print('--------------------------------')
    for i, iou in enumerate(totaliou):
        labelName = label[i]
        iouStr = f'{iou:>5.3f}'
        niouStr = 'empty'
        print('{:<14}: '.format(labelName) + iouStr + '    ' + niouStr)
    print('--------------------------------')
    print(f'Score Average : {(np.sum(totaliou) / 2):>5.3f}' + '    ' + niouStr)

In [ ]:
printClassScores(totaliou)